# Fuzzy grouping
Copyright (c) Microsoft Corporation. All rights reserved.<br>
Licensed under the MIT License.<br>

DataPrep also provides a way to canonicalize strings in a column.


In [1]:
import azureml.dataprep as dprep

In [2]:
dataflow = dprep.read_json(path='./data/json.json', )
dataflow.head(5)

,inspections.business.business_id,inspections.business.name,inspections.business.address,inspections.business.city,inspections.business.postal_code,inspections.business.latitude,inspections.business.longitude,inspections.business.phone_number,inspections.business.TaxCode,inspections.business.business_certificate,inspections.business.application_date,inspections.business.owner_name,inspections.business.owner_address,inspections.Score,inspections.date,inspections.type,inspections.violations
0,16162,Quick-N-Ezee Indian Foods,3861 24th St,SF,94114,,,,H34,467114,May 9 2005 12:00AM,Jagpreet Enterprises,23682 Clawiter Road\n Hayward\n CA\n 94545,100,20130223,Routine - Unscheduled,[]
1,69707,Little Green Cyclo 2,Off The Grid,,,,,,H79,453248,Jul 12 2012 12:00AM,LITTLEGREENCYCLO LLC,"100 Esplanade Ave., Apt. 99\n Pacifica\n CA\n ...",93,20130224,Routine - Unscheduled,"[{""description"":""103112: No hot water or runni..."
2,67565,King of Thai Noodles Cafe,1541 TARAVAL St,SAN FRANCISCO,94116,37.7427,-122.483,,H25,,Oct 12 2011 12:00AM,"Royal Thai Noodles, Inc",2410 19th Ave\n SF\n CA\n 94116,79,20130225,Routine - Unscheduled,"[{""description"":""103139: Improper food storage..."
3,67565,King of Thai Noodles Cafe,1541 TARAVAL St,SAN FRANCISCO,94116,37.7427,-122.483,,H25,,Oct 12 2011 12:00AM,"Royal Thai Noodles, Inc",2410 19th Ave\n SF\n CA\n 94116,,20130225,Complaint,"[{""description"":""103139: Improper food storage..."
4,68701,Grindz,832 Clement St,SF,94118,37.7828,-122.468,,H25,467498,Mar 16 2012 12:00AM,"Ono Grindz, LLC",1055 Granada St.\n Vallejo\n CA\n 94591,100,20130225,Routine - Unscheduled,[]


For a simple case, where you just want to add a column with values brought to the automatically detected cacnonical form, you can call fuzzy_group_column() on an existing dataflow:

In [3]:
dataflow2 = dataflow.fuzzy_group_column(source_column='inspections.business.city',
                                        new_column_name='city_grouped',
                                        similarity_threshold=0.8,
                                        similarity_score_column_name='similarity_score')
dataflow2.head(100)

,inspections.business.business_id,inspections.business.name,inspections.business.address,inspections.business.city,city_grouped,similarity_score,inspections.business.postal_code,inspections.business.latitude,inspections.business.longitude,inspections.business.phone_number,inspections.business.TaxCode,inspections.business.business_certificate,inspections.business.application_date,inspections.business.owner_name,inspections.business.owner_address,inspections.Score,inspections.date,inspections.type,inspections.violations
0,16162,Quick-N-Ezee Indian Foods,3861 24th St,SF,San Francisco,0.814806,94114,,,,H34,467114,May 9 2005 12:00AM,Jagpreet Enterprises,23682 Clawiter Road\n Hayward\n CA\n 94545,100,20130223,Routine - Unscheduled,[]
1,69707,Little Green Cyclo 2,Off The Grid,,,NaN,,,,,H79,453248,Jul 12 2012 12:00AM,LITTLEGREENCYCLO LLC,"100 Esplanade Ave., Apt. 99\n Pacifica\n CA\n ...",93,20130224,Routine - Unscheduled,"[{""description"":""103112: No hot water or runni..."
2,67565,King of Thai Noodles Cafe,1541 TARAVAL St,SAN FRANCISCO,San Francisco,1.000000,94116,37.7427,-122.483,,H25,,Oct 12 2011 12:00AM,"Royal Thai Noodles, Inc",2410 19th Ave\n SF\n CA\n 94116,79,20130225,Routine - Unscheduled,"[{""description"":""103139: Improper food storage..."
3,67565,King of Thai Noodles Cafe,1541 TARAVAL St,SAN FRANCISCO,San Francisco,1.000000,94116,37.7427,-122.483,,H25,,Oct 12 2011 12:00AM,"Royal Thai Noodles, Inc",2410 19th Ave\n SF\n CA\n 94116,,20130225,Complaint,"[{""description"":""103139: Improper food storage..."
4,68701,Grindz,832 Clement St,SF,San Francisco,0.814806,94118,37.7828,-122.468,,H25,467498,Mar 16 2012 12:00AM,"Ono Grindz, LLC",1055 Granada St.\n Vallejo\n CA\n 94591,100,20130225,Routine - Unscheduled,[]
5,69186,"Premier Catering & Events, Inc.",1255 22nd St,S.F.,San Francisco,0.814806,94107,,,14155530288,H30,362812,Apr 30 2012 12:00AM,"Premier Catering & Events, Inc.",298 Magellan Ave.\n SF\n CA\n 94116,,20130225,Reinspection/Followup,[]
6,2689,THE BLUE PLATE,3218 MISSION St,SF,San Francisco,0.814806,94110,37.7452,-122.42,14155286777,H25,325714,,BLUE ENCLAVE LLC,3218 MISSION ST.\n SAN FRANCISCO\n CA\n 94110,98,20130225,Routine - Unscheduled,"[{""description"":""103143: Inadequate warewashin..."
7,15806,Vital Tea Leaf,1044 Grant Ave,San Francisco,San Francisco,1.000000,94133,37.7966,-122.407,,H24,388301,May 23 2005 12:00AM,Minh H. Duong,1044 Grant Ave\n San Francisco\n CA\n 94133,98,20130225,Routine - Unscheduled,"[{""description"":""103157: Food safety certifica..."
8,21807,The Front Porch,65 29th St A,SF,San Francisco,0.814806,94110,37.7439,-122.422,,H25,398500,Jun 7 2006 12:00AM,Front Porch Restaurant LLC,65A 29th Street\n SF\n CA\n 94110,,20130225,Reinspection/Followup,[]
9,69041,Washington Cafe,826 Washington St,San Francisco,San Francisco,1.000000,94108,37.7951,-122.407,,H26,468548,Apr 18 2012 12:00AM,"Washington Caf�, Inc. / Louis Kuang",333 Third Avenue\n Daly City\n CA\n 94014,65,20130225,Routine - Unscheduled,"[{""description"":""103120: Moderate risk food ho..."


Out of the arguments only source_column and new_column_name are required. 
similarity_threshold, if provided controls how close values should be to warrant inclusion in the same fuzzy group.
similarity_score_column_name controls the name of an additional column with the value representing similarity score between original value and detected canonical value. If no similarity_score_column_name is passed in, no column with the score would be added.

As you can see above, couple of ways San Francisco was present in the data were all normalized to 'San Francisco'.
But what if you want more control over what gets grouped, what doesn't and what the canonical value should be? 
The way to get there is by going through a specific builder for FuzzyGoups:

In [4]:
builder = dataflow.builders.fuzzy_group_column(source_column='inspections.business.city',
                                        new_column_name='city_grouped',
                                        similarity_threshold=0.8,
                                        similarity_score_column_name='similarity_score')

In [5]:
builder.learn()
builder.groups

[{'canonicalValue': 'San Francisco',
  'duplicates': [{'similarityScore': 1.0,
    'duplicateValue': 'San Francisco',
    'useForReplacement': True},
   {'similarityScore': 1.0,
    'duplicateValue': 'SAN FRANCISCO',
    'useForReplacement': True},
   {'similarityScore': 0.8148061037063599,
    'duplicateValue': 'SF',
    'useForReplacement': True},
   {'similarityScore': 0.8148061037063599,
    'duplicateValue': 'S.F.',
    'useForReplacement': True}]}]

Here we can see that DataPrep has detected 1 group with 4 values that all map to 'San Francisco' as canonical value.
We can see what kind of effect changing the similarity threshold has:

In [6]:
builder.similarity_threshold = 0.9
builder.learn()
builder.groups

[{'canonicalValue': 'SF',
  'duplicates': [{'similarityScore': 1.0,
    'duplicateValue': 'SF',
    'useForReplacement': True},
   {'similarityScore': 0.9523809552192688,
    'duplicateValue': 'S.F.',
    'useForReplacement': True}]},
 {'canonicalValue': 'San Francisco',
  'duplicates': [{'similarityScore': 1.0,
    'duplicateValue': 'San Francisco',
    'useForReplacement': True},
   {'similarityScore': 1.0,
    'duplicateValue': 'SAN FRANCISCO',
    'useForReplacement': True}]}]

Now that we want similarity of at least 0.9 DataPrep has generated two distinct fuzzy groups.

Let's tweak some of the detected groups before completing the builder and getting our resulting dataflow.

In [7]:
builder.similarity_threshold = 0.8
builder.learn()
groups = builder.groups
groups

[{'canonicalValue': 'San Francisco',
  'duplicates': [{'similarityScore': 1.0,
    'duplicateValue': 'San Francisco',
    'useForReplacement': True},
   {'similarityScore': 1.0,
    'duplicateValue': 'SAN FRANCISCO',
    'useForReplacement': True},
   {'similarityScore': 0.8148061037063599,
    'duplicateValue': 'SF',
    'useForReplacement': True},
   {'similarityScore': 0.8148061037063599,
    'duplicateValue': 'S.F.',
    'useForReplacement': True}]}]

In [8]:
groups[0]['canonicalValue'] = 'SANFRAN'
duplicates = groups[0]['duplicates']
duplicates = duplicates[:-1]
groups[0]['duplicates'] = duplicates
builder.groups = groups
builder.groups

[{'canonicalValue': 'SANFRAN',
  'duplicates': [{'similarityScore': 1.0,
    'duplicateValue': 'San Francisco',
    'useForReplacement': True},
   {'similarityScore': 1.0,
    'duplicateValue': 'SAN FRANCISCO',
    'useForReplacement': True},
   {'similarityScore': 0.8148061037063599,
    'duplicateValue': 'SF',
    'useForReplacement': True}]}]

As you can see, we have modified canonical value to be used for the single fuzzy group as well as removed 'S.F.' from this group's duplicates list.

You can mutate the copy of the list you got from the Builder (be careful to keep the structure of object inside this list) to get desired groups and then update the builder with it.

Now we can get a dataflow with the FuzzyGroup step in it.

In [9]:
dataflow = builder.to_dataflow()

df = dataflow.to_pandas_dataframe()
df

,inspections.business.business_id,inspections.business.name,inspections.business.address,inspections.business.city,city_grouped,similarity_score,inspections.business.postal_code,inspections.business.latitude,inspections.business.longitude,inspections.business.phone_number,inspections.business.TaxCode,inspections.business.business_certificate,inspections.business.application_date,inspections.business.owner_name,inspections.business.owner_address,inspections.Score,inspections.date,inspections.type,inspections.violations
0,16162,Quick-N-Ezee Indian Foods,3861 24th St,SF,SANFRAN,0.814806,94114,,,,H34,467114,May 9 2005 12:00AM,Jagpreet Enterprises,23682 Clawiter Road\n Hayward\n CA\n 94545,100,20130223,Routine - Unscheduled,[]
1,69707,Little Green Cyclo 2,Off The Grid,,,NaN,,,,,H79,453248,Jul 12 2012 12:00AM,LITTLEGREENCYCLO LLC,"100 Esplanade Ave., Apt. 99\n Pacifica\n CA\n ...",93,20130224,Routine - Unscheduled,"[{""description"":""103112: No hot water or runni..."
2,67565,King of Thai Noodles Cafe,1541 TARAVAL St,SAN FRANCISCO,SANFRAN,1.000000,94116,37.7427,-122.483,,H25,,Oct 12 2011 12:00AM,"Royal Thai Noodles, Inc",2410 19th Ave\n SF\n CA\n 94116,79,20130225,Routine - Unscheduled,"[{""description"":""103139: Improper food storage..."
3,67565,King of Thai Noodles Cafe,1541 TARAVAL St,SAN FRANCISCO,SANFRAN,1.000000,94116,37.7427,-122.483,,H25,,Oct 12 2011 12:00AM,"Royal Thai Noodles, Inc",2410 19th Ave\n SF\n CA\n 94116,,20130225,Complaint,"[{""description"":""103139: Improper food storage..."
4,68701,Grindz,832 Clement St,SF,SANFRAN,0.814806,94118,37.7828,-122.468,,H25,467498,Mar 16 2012 12:00AM,"Ono Grindz, LLC",1055 Granada St.\n Vallejo\n CA\n 94591,100,20130225,Routine - Unscheduled,[]
5,69186,"Premier Catering & Events, Inc.",1255 22nd St,S.F.,S.F.,NaN,94107,,,14155530288,H30,362812,Apr 30 2012 12:00AM,"Premier Catering & Events, Inc.",298 Magellan Ave.\n SF\n CA\n 94116,,20130225,Reinspection/Followup,[]
6,2689,THE BLUE PLATE,3218 MISSION St,SF,SANFRAN,0.814806,94110,37.7452,-122.42,14155286777,H25,325714,,BLUE ENCLAVE LLC,3218 MISSION ST.\n SAN FRANCISCO\n CA\n 94110,98,20130225,Routine - Unscheduled,"[{""description"":""103143: Inadequate warewashin..."
7,15806,Vital Tea Leaf,1044 Grant Ave,San Francisco,SANFRAN,1.000000,94133,37.7966,-122.407,,H24,388301,May 23 2005 12:00AM,Minh H. Duong,1044 Grant Ave\n San Francisco\n CA\n 94133,98,20130225,Routine - Unscheduled,"[{""description"":""103157: Food safety certifica..."
8,21807,The Front Porch,65 29th St A,SF,SANFRAN,0.814806,94110,37.7439,-122.422,,H25,398500,Jun 7 2006 12:00AM,Front Porch Restaurant LLC,65A 29th Street\n SF\n CA\n 94110,,20130225,Reinspection/Followup,[]
9,69041,Washington Cafe,826 Washington St,San Francisco,SANFRAN,1.000000,94108,37.7951,-122.407,,H26,468548,Apr 18 2012 12:00AM,"Washington Caf�, Inc. / Louis Kuang",333 Third Avenue\n Daly City\n CA\n 94014,65,20130225,Routine - Unscheduled,"[{""description"":""103120: Moderate risk food ho..."
